In [ ]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
audio_dataset_path = "/content/drive/MyDrive/Deep Learning Lab/DL Project/Data/genres_original"
metadata = pd.read_csv("/content/drive/MyDrive/Deep Learning Lab/DL Project/Data/features_3_sec.csv")
metadata.head()

In [ ]:
def features_extractor(file):
  audio, sample_rate = librosa. load(file_name, res_type='kaiser_fast')
  mfccs_features = librosa. feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
  mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

  return mfccs_scaled_features

In [ ]:
metadata.drop(labels=552, axis=0, inplace=True)

In [ ]:
from tqdm import tqdm
### Now we iterate through every audio file and extract features
### using Mel-Frequency Cepstral Coefficients

extracted_features=[]
for index_num, row in tqdm(metadata.iterrows()):
    try:
        final_class_labels=row["label"]
        file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/',str(row["filename"]))
        data=features_extractor(file_name)
        extracted_features.append([data, final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features, columns=['feature', 'class'])
extracted_features_df.head()

In [ ]:
extracted_features_df ['class' ]. value_counts ()

In [ ]:
X=np.array(extracted_features_df['feature']. tolist ())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X. shape

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y. shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
num_labels=y. shape[1]

In [ ]:
y. shape [1]

In [ ]:
model=Sequential()
model.add(Dense(1024,input_shape=(40,), activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))

###final layer
model.add(Dense(num_labels, activation="softmax") )

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy' , metrics=['accuracy' ], optimizer='adam' )

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 100
num_batch_size = 32

start = datetime.now()

history = model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

In [ ]:
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {train_acc * 100:.2f}%")
print(f"Final Validation Accuracy: {val_acc * 100:.2f}%")

In [ ]:
model.evaluate(X_test,y_test, verbose=0)

In [ ]:
import matplotlib.pyplot as plt

# Plot Loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
plt.title('Model Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy', color='green')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
plt.title('Model Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Get the predicted probabilities
predicted_probabilities = model.predict(X_test)

# Get the predicted class index for each sample
predicted_classes = np.argmax(predicted_probabilities, axis=1)

# Print the predicted classes
print(predicted_classes)

In [ ]:
filename="/content/drive/MyDrive/Deep Learning Lab/DL Project/Data/genres_original/jazz/jazz.00000.wav"
audio, sample_rate = librosa. load(filename, res_type='kaiser_fast')
mfccs_features = librosa. feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features. reshape(1,-1)
print (mfccs_scaled_features)
print(mfccs_scaled_features.shape)

# Get the predicted probabilities
predicted_probabilities = model.predict(mfccs_scaled_features)

# Get the predicted class index
predicted_label = np.argmax(predicted_probabilities, axis=1) # Get the index of the class with highest probability
print (predicted_label)

prediction_class = labelencoder.inverse_transform(predicted_label)
prediction_class

In [ ]:
import pickle

with open('/content/drive/MyDrive/Deep Learning Lab/DL Project/label_encoder.pkl', 'wb') as f:
    pickle.dump(labelencoder, f)

In [ ]:
model.save("/content/drive/MyDrive/Deep Learning Lab/DL Project/best_model.h5")